# Problem 4.a. - Vehicle Routing Problem

In [1]:
import pandas as pd
import os
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
os.chdir(r"C:\Users\User\Desktop\BUS 730 - Prescriptive Modeling & Optimization for BA\Assignments\HW3_Integer (MIP) Optimization") 
%pprint

Pretty printing has been turned OFF


In [2]:
# reading in distancematrix data (one-way travel times between different branch locations (in minutes))
distancedata = pd.read_excel(open('P4_VRPtraveltimes.xlsx','rb'), sheet_name='distancematrix') 
distancedata

,Unnamed: 0,T4X 0B6,T5A 5C1,T5B 0S1,T5E 4C6,T5J 1V7,T5J 3N3,T5K 0M8,T5L 4Z6,T5M 2L7,...,T7X 2K6,T7Z 2W7,T8A 4N5,T8A 5V9,T8H 1S8,T8L 3W2,T8N 3L3,T8N 5T8,T8R 1R4,T9E 6Z7
0,T4X 0B6,0,33,34,34,35,36,37,40,41,...,41,48,24,29,26,45,45,39,54,20
1,T5A 5C1,29,0,14,14,22,24,24,20,21,...,37,42,15,17,14,25,27,22,33,36
2,T5B 0S1,31,15,0,7,10,13,16,15,13,...,31,35,17,18,15,31,27,22,36,38
3,T5E 4C6,31,16,8,0,14,14,14,9,11,...,29,33,17,19,16,30,24,19,30,38
4,T5J 1V7,36,25,13,13,0,8,10,22,14,...,38,42,23,29,23,41,35,29,43,37
5,T5J 3N3,31,25,14,14,4,0,5,19,10,...,34,40,20,25,20,41,32,26,44,32
6,T5K 0M8,34,27,18,15,7,3,0,19,9,...,34,40,23,28,23,44,30,24,43,34
7,T5L 4Z6,39,20,15,9,22,19,16,0,13,...,28,33,25,26,23,32,19,13,32,39
8,T5M 2L7,36,23,15,11,13,11,8,14,0,...,29,34,25,26,23,39,24,19,37,36
9,T5M 3L7,34,24,17,13,14,12,9,12,5,...,24,29,26,28,25,37,23,17,36,35


In [3]:
# converting distancedata to a list of lists (for distance_matrix input)
distmatrix = distancedata.loc[:, 'T4X 0B6':'T9E 6Z7'].values.tolist()
distmatrix[0]  # display the first row of distmatrix

[0, 33, 34, 34, 35, 36, 37, 40, 41, 38, 28, 35, 33, 37, 28, 29, 28, 24, 23, 18, 26, 23, 36, 19, 18, 41, 48, 24, 29, 26, 45, 45, 39, 54, 20]

In [4]:
# converting branch locations to a list
branch = distancedata.iloc[:,0].tolist()
print(branch)

# The vehicles start and end their route at location “T5B 0S1”
depotindex = branch.index("T5B 0S1") # the index number of “T5B 0S1”
depotindex

['T4X 0B6', 'T5A 5C1', 'T5B 0S1', 'T5E 4C6', 'T5J 1V7', 'T5J 3N3', 'T5K 0M8', 'T5L 4Z6', 'T5M 2L7', 'T5M 3L7', 'T5T 1K8', 'T5T 5L5', 'T5Y 0L2', 'T5Z 3J8', 'T6B 0P2', 'T6E 2A3', 'T6E 5A7', 'T6H 5T1', 'T6J 6P9', 'T6K 4B4', 'T6R 0G4', 'T6T 0C2', 'T6V 1J6', 'T6W 1A2', 'T6X 0P2', 'T7X 2K6', 'T7Z 2W7', 'T8A 4N5', 'T8A 5V9', 'T8H 1S8', 'T8L 3W2', 'T8N 3L3', 'T8N 5T8', 'T8R 1R4', 'T9E 6Z7']


2

In [5]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distmatrix
    data['num_vehicles'] = 6
    data['depot'] = depotindex
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id+1)
        route_distance = 0        
        while not routing.IsEnd(index):
            branchLoc = branch[manager.IndexToNode(index)]
            plan_output += ' {} -> '.format(branchLoc)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        branchLoc = branch[manager.IndexToNode(index)]
        plan_output += '{}\n'.format(branchLoc)
        plan_output += 'Travel time of the route: {} minutes\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route travel time: {} minutes\n'.format(max_route_distance))

    
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [branch[manager.IndexToNode(index)]]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            branchLoc = branch[manager.IndexToNode(index)]
            route.append(branchLoc)
        routes.append(route)
    return routes


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(1000)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

    routes = get_routes(solution, routing, manager)
    
    # Display the routes.
    for i, route in enumerate(routes):
        print('Route', i+1, route)

if __name__ == '__main__':
    main()

Route for vehicle 1:
 T5B 0S1 ->  T5Z 3J8 ->  T8R 1R4 ->  T8N 3L3 ->  T8N 5T8 ->  T6V 1J6 ->  T5L 4Z6 ->  T5E 4C6 -> T5B 0S1
Travel time of the route: 94 minutes

Route for vehicle 2:
 T5B 0S1 ->  T8A 4N5 ->  T8H 1S8 ->  T8A 5V9 ->  T8L 3W2 ->  T5Y 0L2 ->  T5A 5C1 -> T5B 0S1
Travel time of the route: 98 minutes

Route for vehicle 3:
 T5B 0S1 ->  T5J 1V7 ->  T5K 0M8 ->  T5J 3N3 ->  T6E 2A3 ->  T6H 5T1 ->  T6E 5A7 ->  T6B 0P2 -> T5B 0S1
Travel time of the route: 75 minutes

Route for vehicle 4:
 T5B 0S1 ->  T5M 2L7 ->  T5M 3L7 ->  T6R 0G4 ->  T6J 6P9 ->  T6W 1A2 ->  T6X 0P2 ->  T6K 4B4 -> T5B 0S1
Travel time of the route: 97 minutes

Route for vehicle 5:
 T5B 0S1 ->  T4X 0B6 ->  T9E 6Z7 ->  T6T 0C2 -> T5B 0S1
Travel time of the route: 98 minutes

Route for vehicle 6:
 T5B 0S1 ->  T7Z 2W7 ->  T7X 2K6 ->  T5T 5L5 ->  T5T 1K8 -> T5B 0S1
Travel time of the route: 97 minutes

Maximum of the route travel time: 98 minutes

Route 1 ['T5B 0S1', 'T5Z 3J8', 'T8R 1R4', 'T8N 3L3', 'T8N 5T8', 'T6V 1J6

### Interpretation of the solution

The optimal solution is to minimize the travel time of the longest single route among all vehicles. The goal is to complete all deliveries (visit all branch locations) as soon as possible. In this case, the travel time of the longest single route is 98 minutes under the optimal routes, which is the travel time for vehicle 5 route (Route 5 above).  

The optimal routes for 6 vehicles are shown in above solution output.